# WayNet - Road Segmentation Task Project

### Dependencies

In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 3.5 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-community 0.3.1 requires langchain<0.4.0,>=0.3.1, but you have langchain 0.0.154 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
langchain 0.0.154 requires pydantic<2,>=1, but 

In [2]:
import cv2
import os
import matplotlib.pyplot as plt
from datasets import load_dataset
import tensorflow as tf

### Loading the Data

In [3]:
df_train = load_dataset("bnsapa/road-detection")
df_train

README.md:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

(…)-00000-of-00001-5f0e1c478e60fe5a.parquet:   0%|          | 0.00/72.3M [00:00<?, ?B/s]

(…)-00000-of-00001-624fa772f83c1b9f.parquet:   0%|          | 0.00/8.75M [00:00<?, ?B/s]

(…)-00000-of-00001-7a98cb311c8d7ff4.parquet:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'segment', 'lane'],
        num_rows: 160
    })
    test: Dataset({
        features: ['image', 'segment', 'lane'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['image', 'segment', 'lane'],
        num_rows: 20
    })
})